In [ ]:
import os, shutil, requests
import osmnx as ox
import pandas as pd
import geopandas as gpd
import 

## Fetch the data
* Fetch DCFC charging stations (from NREL API)

In [ ]:
#Get the key
key = open('Data/NRELkey.txt','r').readline()

In [ ]:
#Construct the service request
baseURL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?'
params = {
    'api_key':key,                #Our key, to authenticate the service
    'status':'E',                 #Request only currently open sites 
    'fuel_type':'ELEC',           #Request only Electric sites
    'ev_charging_level':'dc_fast',#Request only DCFC sites
    'state':'NC' ,                #Request only NC sites
    'access':'public',            #Request only Public sites
    'groups_with_access_code':'Public', #Request only sites open to Public
    'limit':'all'                 #Don't cap results to first 200 (default)
}

In [ ]:
#Execute the request, converting the response to a dataframe
df_ALL = pd.DataFrame.from_dict(requests.get(baseURL,params).json()['fuel_stations'])

In [ ]:
#Drop excess columns
dropCols = [col for col in df_ALL.columns if col[:3] in ('bd_','cng','hy_','ng_','e85','lng','lpg')]
dropCols += [col for col in df_ALL.columns if col[-2:] == 'fr']
df_ALL.drop(columns = dropCols,inplace=True)

In [ ]:
#Remove sites in the Tesla network
df_DCFC = df_ALL.loc[df_ALL['ev_network'] != 'Tesla']

In [ ]:
#Save to a csv
df_DCFC.to_csv('Data/NREL_DCFCsites.csv',index=False)

In [ ]:
#Convert to a geodataframe
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_DCFC.longitude, df_DCFC.latitude)]
gdf_DCFC = gpd.GeoDataFrame(df_DCFC,geometry=geometry,crs='epsg:4326')

In [ ]:
gdf_DCFC.plot()

* Read the NC Highways graphml into a graph object; fetch if not present

In [ ]:
#Fetch the NC highway graph
graph_filename = 'Data/NC_highways.graphml'
if os.path.exists(graph_filename):
    print(f"Loading graph from {graph_filename}")
    nc_graph = ox.load_graphml(filename=os.path.basename(graph_filename),
                               folder=os.path.dirname(graph_filename))
else:
    print("Fetching data. This may take a while...")
    nc_graph = ox.graph_from_place("North Carolina",
                                    network_type='drive',
                                    simplify=True,
                                    retain_all=False,
                                    truncate_by_edge=True,
                                    infrastructure='way["highway"~"motorway"]')
    print(f"Saving file to {graph_filename}")
    ox.save_graphml(nc_graph,
                    filename=os.path.basename(graph_filename),
                    folder=os.path.dirname(graph_filename))
                    

In [ ]:
#Convert graph to a geodataframes
gdf_nodes, gdf_edges = ox.graph_to_gdfs(nc_graph)

In [ ]:
#Projections!
from pyproj import CRS

#Project to UTM (for analysis)
utm17N_prj = CRS.from_epsg(32617)
gdf_edges_utm = gdf_edges.to_crs(utm17N_prj)
gdf_DCFC_utm = gdf_DCFC.to_crs(utm17N_prj)

#Project to Web Mercator (for plotting)
wm_prj = CRS.from_epsg(3857)
gdf_edges_wm = gdf_edges.to_crs(wm_prj)
gdf_DCFC_wm = gdf_DCFC.to_crs(wm_prj)

## Visualize the data

In [ ]:
import contextily as ctx

In [ ]:
#Plot the data
ax = gdf_edges_wm.plot(figsize=(20,10))
gdf_DCFC_wm.plot(color='red',ax=ax)
ctx.add_basemap(ax)